In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [3]:
class TopicClassifier:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def classify_topic(self, topic, information):
        """
        Classify podcast topic and determine optimal approach
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            
        Returns:
            JSON with classification results
        """
        
        prompt = f"""
You are an expert in analyzing and classifying topics for producing natural and engaging Arabic podcasts.

Task: Analyze the following topic and determine the best approach for presenting it in an Arabic podcast.

Topic: {topic}
Background Information: {information}

Analyze the topic and return the result in JSON format containing:

{{
    "primary_category": "Main category",
    "category_justification": "Reason for choosing this category based on the topic's nature",
    "optimal_style": "Optimal discussion style",
    "discourse_pattern": "Appropriate discourse pattern",
    "audience_engagement_goal": "Audience engagement objective",
    "cultural_sensitivity_level": "Cultural sensitivity level",
    "controversy_potential": "Controversy potential",
    "key_discussion_angles": [
        "Main expected discussion angles",
        "Points that will interest the Arabic audience"
    ],
    "natural_tension_points": [
        "Natural tension points in the topic",
        "Aspects that might generate healthy debate"
    ],
    "cultural_connection_opportunities": [
        "Opportunities to connect with Arabic culture",
        "Relevant local and regional references"
    ]
}}

Available Categories:
1. "العلوم والتكنولوجيا" - For technical, scientific topics and innovations
2. "السياسة والشؤون العامة" - For political topics, current events and public affairs
3. "القضايا الاجتماعية" - For social topics, relationships, values and social challenges
4. "الرياضة والترفيه" - For sports, arts and entertainment topics
5. "التاريخ والثقافة" - For historical, heritage and cultural topics

Available Styles:
- "حواري" - Natural and friendly dialogue between host and guest
- "تعليمي" - Focus on explanation and education in an entertaining way
- "ترفيهي" - Fun and light with humorous touches
- "تحليلي" - Deep, specialized and analytical discussion

Discourse Patterns:
- "رسمي" - Formal and respectful language
- "ودي" - Warm and familiar language
- "جدلي" - Lively discussion with multiple viewpoints
- "سردي" - Storytelling and narrative style

Cultural Sensitivity Level:
- "عالي" - Requires extreme caution in handling
- "متوسط" - Needs moderate cultural consideration
- "منخفض" - Generally acceptable topic

Controversy Potential:
- "عالية" - Inherently controversial topic
- "متوسطة" - May generate some disagreements
- "منخفضة" - Generally acceptable topic

Important Instructions:
- Analyze the topic deeply, not superficially
- Consider the Arabic cultural context in your analysis
- Focus on what makes the topic appealing to Arabic audiences
- Ensure the classification serves natural and spontaneous content production
- Do not include ```json markers at the beginning or end
- Return only valid JSON without any additional text
- All JSON values should be written in Modern Standard Arabic (MSA)
- The optimal episode duration is 10 minutes
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in analyzing and classifying topics for producing professional Arabic podcasts. You specialize in understanding Arabic audiences and their interests. Always respond with JSON values in Modern Standard Arabic."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent classification
        )
        
        return response.choices[0].message.content

In [4]:
# Testing Instructions:

# To test Step 1, add this to a new cell in your notebook:

# Test Step 1: Topic Classification
classifier = TopicClassifier(deployment, "gpt-4o")

# Test with the singlehood topic
topic = "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي"

information = '''
مع انتشار تقنيات الذكاء الاصطناعي بسرعة في العالم العربي، تزداد المخاوف حول تأثيرها على الهوية الثقافية واللغة العربية. 
تشير الدراسات إلى أن 78% من المحتوى الرقمي باللغة الإنجليزية، بينما المحتوى العربي لا يتجاوز 3%. 
معظم نماذج الذكاء الاصطناعي الحالية مدربة على بيانات غربية، مما يثير تساؤلات حول قدرتها على فهم السياق الثقافي العربي.
في المقابل، تسعى دول مثل الإمارات والسعودية لتطوير نماذج ذكاء اصطناعي عربية مثل "جايس" و"الحوراء" لمواجهة هذا التحدي.
التحدي الأكبر يكمن في كيفية الاستفادة من هذه التقنيات لتعزيز الثقافة العربية بدلاً من تهميشها، وضمان أن تخدم الذكاء الاصطناعي قيمنا ومبادئنا.
'''

# Run classification
classification_result = classifier.classify_topic(topic, information)
print("Classification Result:")
print(classification_result)



# Parse and examine the JSON
try:
    parsed_result = json.loads(classification_result)
    print(f"\nPrimary Category: {parsed_result['primary_category']}")
    print(f"Optimal Style: {parsed_result['optimal_style']}")
    print(f"Emotional Intent: {parsed_result['emotional_intent']}")
    print(f"Discourse Pattern: {parsed_result['discourse_pattern']}")
except:
    print("Error parsing JSON result")

Classification Result:
{
    "primary_category": "التاريخ والثقافة",
    "category_justification": "الموضوع يتناول الهوية الثقافية العربية وتأثير الذكاء الاصطناعي عليها، مما يجعله مرتبطًا بالتراث والثقافة العربية بشكل مباشر.",
    "optimal_style": "تحليلي",
    "discourse_pattern": "رسمي",
    "audience_engagement_goal": "إثارة التفكير حول أهمية الحفاظ على الهوية الثقافية في ظل التطورات الرقمية، وتشجيع الجمهور على المشاركة في النقاش حول مستقبل الثقافة العربية.",
    "cultural_sensitivity_level": "عالي",
    "controversy_potential": "متوسطة",
    "key_discussion_angles": [
        "تأثير الذكاء الاصطناعي على اللغة العربية والهوية الثقافية.",
        "جهود الدول العربية في تطوير نماذج ذكاء اصطناعي تعكس الثقافة المحلية.",
        "كيفية تحقيق التوازن بين الابتكار الرقمي والحفاظ على القيم الثقافية."
    ],
    "natural_tension_points": [
        "هيمنة المحتوى الرقمي باللغة الإنجليزية مقابل ضعف المحتوى العربي.",
        "التحديات في تدريب نماذج ذكاء اصطناعي على بيانات عربية ذات جودة عالية.

In [5]:
class SimplePersonaGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_personas(self, topic, information, classification_result):
        """
        Generate simple but effective host and guest personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            
        Returns:
            JSON with simple host and guest personas
        """
        
        # Parse classification to understand the requirements
        try:
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid classification JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
You are an expert in designing Arabic podcast personas.

Task: Create simple and suitable host and guest personas for this topic.

Topic: {topic}
Information: {information}
Category: {primary_category}
Required Style: {optimal_style}

Return the result in simple JSON format:

{{
    "host": {{
        "name": "Host's name",
        "age": numeric_age,
        "background": "Brief background in one sentence",
        "personality": "Personality description in one sentence",
        "speaking_style": "Speaking style in one sentence"
    }},
    "guest": {{
        "name": "Guest's name", 
        "age": numeric_age,
        "background": "Brief background in one sentence",
        "expertise": "Area of expertise in one sentence",
        "personality": "Personality description in one sentence",
        "speaking_style": "Speaking style in one sentence"
    }},
    "why_good_match": "Why this host and guest are suitable for this topic - one sentence"
}}

Requirements:
- Use familiar Arabic names
- Simple and believable personas
- Suitable for the topic and required style
- Host should be curious and guest should be expert or have experience
- All JSON values must be in Modern Standard Arabic (MSA)
- JSON keys should be in English
- Do not include ```json markers
- Return only JSON
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in designing simple and effective podcast personas. Required style: {optimal_style}. Always provide JSON values in Modern Standard Arabic while keeping keys in English."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )
        
        return response.choices[0].message.content

In [6]:
persona = SimplePersonaGenerator(deployment, "gpt-4o")
persona_result = persona.generate_personas(topic, information, classification_result)

In [7]:
class ConversationStructureGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_conversation_structure(self, topic, information, classification_result, personas_result):
        """
        Step 3: Generate core conversation structure (V1 skeleton)
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            
        Returns:
            JSON with V1-style conversation structure (without rich dialogue content)
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided for classification or personas")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        discourse_pattern = classification.get("discourse_pattern", "")
        recommended_duration = classification.get("recommended_duration", "10 دقيقة")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
You are an expert in designing conversation structures for Arabic podcasts.

Task: Create the basic conversation structure in natural dialogue format.

Topic: {topic}
Information: {information}

Context:
- Category: {primary_category}
- Style: {optimal_style}
- Discourse Pattern: {discourse_pattern}
- Duration: {recommended_duration}

Characters:
- Host: {host_name} - {host.get('background', '')}
- Guest: {guest_name} - {guest.get('background', '')}

Create the conversation outline in JSON format:

{{
    "episode_topic": "{topic}",
    "personas": {{
        "host": {{
            "name": "{host_name}",
            "background": "{host.get('background', '')}",
            "speaking_style": "{host.get('speaking_style', '')}"
        }},
        "guest": {{
            "name": "{guest_name}",
            "background": "{guest.get('background', '')}",
            "speaking_style": "{guest.get('speaking_style', '')}"
        }}
    }},
    "conversation_flow": {{
        "intro1": {{
            "opening_line": "Actual opening line by the host",
            "podcast_introduction": "Podcast introduction related to the topic",
            "episode_hook": "Specific engaging sentence about the topic",
            "tone_guidance": "Appropriate tone for {optimal_style} style"
        }},
        "intro2": {{
            "topic_introduction": "Clear topic introduction",
            "guest_welcome": "Welcome to guest {guest_name}",
            "guest_bio_highlight": "Guest background introduction",
            "transition_to_discussion": "Transition to main discussion"
        }},
        "main_discussion": [
            {{
                "point_title": "First main point of the topic",
                "personal_angle": "How it relates to the characters' backgrounds"
            }},
            {{
                "point_title": "Second main point of the topic", 
                "personal_angle": "Personal angle for this point"
            }},
            {{
                "point_title": "Third main point of the topic",
                "personal_angle": "Personal connection and concluding angle"
            }}
        ],
        "closing": {{
            "conclusion": {{
                "main_takeaways": "Main takeaways",
                "guest_final_message": "Guest's final message",
                "host_closing_thoughts": "Host's closing thoughts"
            }},
            "outro": {{
                "guest_appreciation": "Thank the guest",
                "audience_thanks": "Thank the listeners",
                "call_to_action": "Call for engagement",
                "final_goodbye": "Final goodbye"
            }}
        }}
    }},
    "cultural_context": {{
        "proverbs_sayings": [
            "Relevant Arabic proverb for the topic",
            "Related wisdom saying"
        ],
        "regional_references": [
            "Local reference related to the topic",
            "Relevant Arab experience"
        ]
    }},
    "language_style": {{
        "formality_level": "Appropriate level for {optimal_style} style",
        "dialect_touches": "Light dialectal touches according to the host",
        "vocabulary_richness": "Vocabulary suitable for the topic"
    }},
    "technical_notes": {{
        "pacing_guidance": "Appropriate pacing for {recommended_duration} duration",
        "pause_points": "Natural pause points",
        "emphasis_moments": "Important emphasis moments"
    }}
}}

Important Requirements:
- Write actual content, not descriptions
- Use the real character names ({host_name}, {guest_name})
- Make all content related to the topic: {topic}
- Focus on building the basic conversation structure
- Make content ready for development in the next step
- All JSON values must be in Modern Standard Arabic (MSA)
- JSON keys should be in English
- Do not include ```json markers
- Return only valid JSON
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in designing conversation structures for Arabic podcasts. Required style: {optimal_style}. Always provide JSON values in Modern Standard Arabic while keeping keys in English."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6  # Balanced for structure creation
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_conversation_structure(self, structure_json):
        """
        Validate the conversation structure
        """
        required_keys = ["episode_topic", "personas", "conversation_flow", "cultural_context", "language_style", "technical_notes"]
        
        conversation_flow_required = ["intro1", "intro2", "main_discussion", "closing"]
        intro1_required = ["opening_line", "podcast_introduction", "episode_hook"]
        intro2_required = ["topic_introduction", "guest_welcome", "guest_bio_highlight"]
        
        try:
            import json
            structure = json.loads(structure_json)
            
            missing_keys = []
            
            # Check main structure
            for key in required_keys:
                if key not in structure:
                    missing_keys.append(key)
            
            # Check conversation flow
            if "conversation_flow" in structure:
                conv_flow = structure["conversation_flow"]
                for key in conversation_flow_required:
                    if key not in conv_flow:
                        missing_keys.append(f"conversation_flow.{key}")
                
                # Check intro1
                if "intro1" in conv_flow:
                    intro1 = conv_flow["intro1"]
                    for key in intro1_required:
                        if key not in intro1:
                            missing_keys.append(f"intro1.{key}")
                
                # Check intro2
                if "intro2" in conv_flow:
                    intro2 = conv_flow["intro2"]
                    for key in intro2_required:
                        if key not in intro2:
                            missing_keys.append(f"intro2.{key}")
                
                # Check main discussion
                if "main_discussion" in conv_flow:
                    main_disc = conv_flow["main_discussion"]
                    if not isinstance(main_disc, list) or len(main_disc) < 3:
                        missing_keys.append("main_discussion (need at least 3 points)")
                    else:
                        for i, point in enumerate(main_disc):
                            if "point_title" not in point:
                                missing_keys.append(f"main_discussion[{i}].point_title")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Conversation structure validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_structure_quality(self, structure_json):
        """
        Analyze the quality of the conversation structure
        """
        try:
            import json
            structure = json.loads(structure_json)
            
            analysis = {}
            
            # Check completeness
            conv_flow = structure.get("conversation_flow", {})
            analysis["has_intro1"] = bool(conv_flow.get("intro1"))
            analysis["has_intro2"] = bool(conv_flow.get("intro2"))
            analysis["has_main_discussion"] = bool(conv_flow.get("main_discussion"))
            analysis["has_closing"] = bool(conv_flow.get("closing"))
            
            # Check main discussion depth
            main_disc = conv_flow.get("main_discussion", [])
            analysis["discussion_points"] = len(main_disc)
            analysis["adequate_points"] = len(main_disc) >= 3
            
            # Check cultural context
            cultural = structure.get("cultural_context", {})
            analysis["has_proverbs"] = len(cultural.get("proverbs_sayings", [])) >= 1
            analysis["has_regional_refs"] = len(cultural.get("regional_references", [])) >= 1
            
            # Overall readiness
            readiness_indicators = [
                analysis["has_intro1"],
                analysis["has_intro2"],
                analysis["has_main_discussion"],
                analysis["has_closing"],
                analysis["adequate_points"],
                analysis["has_proverbs"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_next_step"] = analysis["readiness_score"] >= 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze structure quality"}

# Usage:
# generator = ConversationStructureGenerator(deployment, "Fanar") 
# structure_result = generator.generate_conversation_structure(topic, information, classification_result, personas_result)

In [8]:
generator = ConversationStructureGenerator(deployment, "gpt-4o")
outline_result = generator.generate_conversation_structure(topic, information, classification_result, persona_result)
print(outline_result)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "ليلى أحمد",
      "background": "صحفية ومقدمة برامج ثقافية مهتمة بالتكنولوجيا والهوية العربية.",
      "speaking_style": "تتحدث بطريقة تحليلية واضحة وتطرح أسئلة ذكية."
    },
    "guest": {
      "name": "د. سامي الحسن",
      "background": "أستاذ جامعي وخبير في الذكاء الاصطناعي وتطبيقاته في العالم العربي.",
      "speaking_style": "يشرح الأفكار بطريقة علمية مبسطة وموجهة للجمهور."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحبًا بكم في حلقة جديدة من بودكاستنا حيث نناقش القضايا التي تؤثر على ثقافتنا وهويتنا.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع في غاية الأهمية: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هناك تساؤلات حول تأثيره على لغتنا وقيمنا. هل يمكن لهذه التقنيات أن تكون أداة لتعزيز هويتنا أم أنها خطر يهد

In [9]:
class DialogueContentEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_dialogue_content(self, topic, information, classification_result, personas_result, structure_result):
        """
        Step 4: Enhance conversation structure with rich dialogue content
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            structure_result: JSON string from Step 3 conversation structure
            
        Returns:
            Enhanced structure with rich dialogue content (V1 style)
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            structure = json.loads(structure_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
You are an expert in enriching conversations with natural dialogue content for Arabic podcasts.

Task: Add rich dialogue content to the existing structure to make it ready for script generation.

Topic: {topic}
Information: {information}

Context:
- Category: {primary_category}
- Style: {optimal_style}
- Cultural Sensitivity: {cultural_sensitivity}

Characters:
- Host: {host_name} - {host.get('personality', '')}
- Guest: {guest_name} - {guest.get('expertise', '')}

Current Structure:
{structure_result}

Add the following elements to the existing structure and return the complete result:

For intro1: Add
- "spontaneity_elements": [list of specific spontaneous phrases]

For intro2: Add  
- "cultural_connections": [specific cultural connection to the topic]

For each point in main_discussion: Add
- "spontaneous_triggers": [spontaneous triggers related to the point]
- "disagreement_points": "potential points of disagreement"
- "cultural_references": [proverbs and cultural references]
- "natural_transitions": "natural transition to the next point"
- "emotional_triggers": "emotional triggers"

Add new section:
"spontaneous_moments": {{
    "natural_interruptions": [natural interruption points],
    "emotional_reactions": [emotional reactions],
    "personal_stories": [personal stories based on character backgrounds],
    "humorous_moments": [funny moments related to the topic]
}},

Add new section:
"personality_interactions": {{
    "host_strengths": "{host_name}'s strengths in dialogue",
    "guest_expertise": "{guest_name}'s areas of expertise",
    "natural_chemistry": "how they interact naturally",
    "tension_points": "healthy tension points",
    "collaboration_moments": "moments of collaboration"
}}

Important Requirements:
- Keep all existing content from the basic structure
- Only add the new content
- Write actual content, not descriptions
- Make every addition related to the topic: {topic}
- Make content suitable for the style: {optimal_style}
- Use character names ({host_name}, {guest_name})
- Make content natural and dialogue-ready
- All JSON values must be in Modern Standard Arabic (MSA)
- JSON keys should be in English
- Do not include ```json markers
- Return complete enhanced JSON
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in enriching conversations with natural dialogue content. Style: {optimal_style}. Always provide JSON values in Modern Standard Arabic while keeping keys in English."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7  # Higher creativity for dialogue content
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_enhanced_content(self, enhanced_json):
        """
        Validate the enhanced dialogue content
        """
        required_new_elements = ["spontaneous_moments", "personality_interactions"]
        
        # Elements that should be added to existing sections
        intro1_should_have = ["spontaneity_elements"]
        intro2_should_have = ["cultural_connections"]
        main_discussion_should_have = ["spontaneous_triggers", "cultural_references", "disagreement_points"]
        
        try:
            import json
            enhanced = json.loads(enhanced_json)
            
            missing_elements = []
            
            # Check new sections
            for element in required_new_elements:
                if element not in enhanced:
                    missing_elements.append(element)
            
            # Check enhanced intro1
            conv_flow = enhanced.get("conversation_flow", {})
            intro1 = conv_flow.get("intro1", {})
            for element in intro1_should_have:
                if element not in intro1:
                    missing_elements.append(f"intro1.{element}")
            
            # Check enhanced intro2
            intro2 = conv_flow.get("intro2", {})
            for element in intro2_should_have:
                if element not in intro2:
                    missing_elements.append(f"intro2.{element}")
            
            # Check enhanced main discussion
            main_disc = conv_flow.get("main_discussion", [])
            for i, point in enumerate(main_disc):
                for element in main_discussion_should_have:
                    if element not in point:
                        missing_elements.append(f"main_discussion[{i}].{element}")
            
            if missing_elements:
                return False, f"Missing enhanced elements: {missing_elements}"
            
            return True, "Dialogue content enhancement validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_enhancement_quality(self, enhanced_json):
        """
        Analyze the quality of dialogue content enhancement
        """
        try:
            import json
            enhanced = json.loads(enhanced_json)
            
            analysis = {}
            
            # Check spontaneous moments
            spont_moments = enhanced.get("spontaneous_moments", {})
            analysis["has_interruptions"] = len(spont_moments.get("natural_interruptions", [])) >= 2
            analysis["has_personal_stories"] = len(spont_moments.get("personal_stories", [])) >= 2
            analysis["has_humor"] = len(spont_moments.get("humorous_moments", [])) >= 1
            
            # Check personality interactions
            personality = enhanced.get("personality_interactions", {})
            analysis["has_chemistry"] = bool(personality.get("natural_chemistry"))
            analysis["has_host_strengths"] = bool(personality.get("host_strengths"))
            analysis["has_guest_expertise"] = bool(personality.get("guest_expertise"))
            
            # Check main discussion enhancement
            conv_flow = enhanced.get("conversation_flow", {})
            main_disc = conv_flow.get("main_discussion", [])
            
            enhanced_points = 0
            for point in main_disc:
                if (point.get("spontaneous_triggers") and 
                    point.get("cultural_references") and 
                    point.get("disagreement_points")):
                    enhanced_points += 1
            
            analysis["enhanced_discussion_points"] = enhanced_points
            analysis["all_points_enhanced"] = enhanced_points == len(main_disc)
            
            # Overall readiness for script generation
            readiness_indicators = [
                analysis["has_interruptions"],
                analysis["has_personal_stories"],
                analysis["has_chemistry"],
                analysis["has_host_strengths"],
                analysis["all_points_enhanced"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_script_generation"] = analysis["readiness_score"] >= 4
            
            return analysis
            
        except:
            return {"error": "Could not analyze enhancement quality"}

# Usage:
# enhancer = DialogueContentEnhancer(deployment, "Fanar")
# enhanced_result = enhancer.enhance_dialogue_content(topic, information, classification_result, personas_result, structure_result)

In [10]:
style_enhancer = DialogueContentEnhancer(deployment, "gpt-4o")
style_enhanced_result = style_enhancer.enhance_dialogue_content(topic, information, classification_result, persona_result, outline_result)
print(style_enhanced_result)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "ليلى أحمد",
      "background": "صحفية ومقدمة برامج ثقافية مهتمة بالتكنولوجيا والهوية العربية.",
      "speaking_style": "تتحدث بطريقة تحليلية واضحة وتطرح أسئلة ذكية."
    },
    "guest": {
      "name": "د. سامي الحسن",
      "background": "أستاذ جامعي وخبير في الذكاء الاصطناعي وتطبيقاته في العالم العربي.",
      "speaking_style": "يشرح الأفكار بطريقة علمية مبسطة وموجهة للجمهور."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحبًا بكم في حلقة جديدة من بودكاستنا حيث نناقش القضايا التي تؤثر على ثقافتنا وهويتنا.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع في غاية الأهمية: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هناك تساؤلات حول تأثيره على لغتنا وقيمنا. هل يمكن لهذه التقنيات أن تكون أداة لتعزيز هويتنا أم أنها خطر يهد

In [11]:
class FinalPolishEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def add_final_polish(self, topic, information, classification_result, personas_result, enhanced_content_result):
        """
        Step 5: Add final polish and spontaneity for natural flow
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            enhanced_content_result: JSON string from Step 4 enhanced content
            
        Returns:
            Final V1-style outline with complete natural flow elements
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            enhanced_content = json.loads(enhanced_content_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
You are an expert in adding final touches to conversations to make them completely natural and spontaneous.

Task: Add final touches and natural flow to the enhanced content.

Topic: {topic}
Information: {information}

Context:
- Category: {primary_category}
- Style: {optimal_style}
- Cultural Sensitivity: {cultural_sensitivity}

Characters:
- Host: {host_name} - {host.get('speaking_style', '')}
- Guest: {guest_name} - {guest.get('speaking_style', '')}

Current Enhanced Content:
{enhanced_content_result}

Add the following final improvements:

1. Enhance "spontaneous_moments" by adding more details:
   - Add more specific "natural_interruptions"
   - Add more realistic "emotional_reactions"
   - Add more "personal_stories" suitable for the characters
   - Add more "humorous_moments" related to the topic

2. Enhance "personality_interactions" with more details:
   - Make "natural_chemistry" more specific
   - Add details to "tension_points"
   - Make "collaboration_moments" clearer

3. Add new section "shared_experiences":
   {{
       "common_ground": [
           "similarities between host and guest",
           "shared experiences they can discuss"
       ],
       "generational_perspectives": [
           "generational differences that may appear in discussion",
           "different viewpoints based on age and experience"
       ]
   }}

4. Add new section "contemporary_relevance":
   {{
       "current_events": [
           "current events related to the topic",
           "recent developments in the field"
       ],
       "future_implications": [
           "future implications of the topic",
           "what can be expected in coming years"
       ]
   }}

5. Enhance "cultural_context" sections with more details:
   - Add more "proverbs_sayings"
   - Add more "regional_references"
   - Make "shared_experiences" more specific

Final Requirements:
- Keep all existing content from Step 4
- Only add the final improvements
- Write specific content, not general
- Make every addition related to the topic: {topic}
- Make characters seem real and interactive
- Use character names ({host_name}, {guest_name})
- Make flow extremely natural
- All JSON values must be in Modern Standard Arabic (MSA)
- JSON keys should be in English
- Do not include ```json markers
- Return complete final JSON
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in adding final touches to natural conversations. Style: {optimal_style}. Always provide JSON values in Modern Standard Arabic while keeping keys in English."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8  # High creativity for final spontaneity
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_final_outline(self, final_json):
        """
        Validate the final polished outline
        """
        required_new_elements = ["shared_experiences", "contemporary_relevance"]
        
        # Enhanced elements that should be richer
        enhanced_elements = {
            "spontaneous_moments": ["natural_interruptions", "emotional_reactions", "personal_stories", "humorous_moments"],
            "personality_interactions": ["natural_chemistry", "tension_points", "collaboration_moments"]
        }
        
        try:
            import json
            final_outline = json.loads(final_json)
            
            missing_elements = []
            
            # Check new sections
            for element in required_new_elements:
                if element not in final_outline:
                    missing_elements.append(element)
            
            # Check enhanced spontaneous moments
            spont_moments = final_outline.get("spontaneous_moments", {})
            for element in enhanced_elements["spontaneous_moments"]:
                if element not in spont_moments:
                    missing_elements.append(f"spontaneous_moments.{element}")
                elif len(spont_moments.get(element, [])) < 2:
                    missing_elements.append(f"spontaneous_moments.{element} (needs at least 2 items)")
            
            # Check enhanced personality interactions
            personality = final_outline.get("personality_interactions", {})
            for element in enhanced_elements["personality_interactions"]:
                if element not in personality:
                    missing_elements.append(f"personality_interactions.{element}")
            
            if missing_elements:
                return False, f"Missing final elements: {missing_elements}"
            
            return True, "Final outline validation successful - ready for V1 script generation"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_final_quality(self, final_json):
        """
        Analyze the quality and completeness of the final outline
        """
        try:
            import json
            final_outline = json.loads(final_json)
            
            analysis = {}
            
            # Check V1 compatibility
            required_v1_sections = ["episode_topic", "personas", "conversation_flow", "spontaneous_moments", "personality_interactions"]
            analysis["v1_compatible"] = all(section in final_outline for section in required_v1_sections)
            
            # Check conversation flow completeness
            conv_flow = final_outline.get("conversation_flow", {})
            analysis["has_complete_flow"] = all(section in conv_flow for section in ["intro1", "intro2", "main_discussion", "closing"])
            
            # Check spontaneous moments richness
            spont_moments = final_outline.get("spontaneous_moments", {})
            spont_count = sum(len(spont_moments.get(key, [])) for key in ["natural_interruptions", "personal_stories", "humorous_moments"])
            analysis["spontaneous_richness"] = spont_count
            analysis["adequate_spontaneity"] = spont_count >= 6
            
            # Check cultural integration depth
            cultural = final_outline.get("cultural_context", {})
            cultural_count = sum(len(cultural.get(key, [])) for key in ["proverbs_sayings", "regional_references", "shared_experiences"])
            analysis["cultural_depth"] = cultural_count
            analysis["adequate_culture"] = cultural_count >= 4
            
            # Check main discussion enhancement
            main_disc = conv_flow.get("main_discussion", [])
            enhanced_points = sum(1 for point in main_disc if all(key in point for key in ["spontaneous_triggers", "cultural_references", "disagreement_points"]))
            analysis["enhanced_discussion_points"] = enhanced_points
            analysis["all_discussions_enhanced"] = enhanced_points == len(main_disc)
            
            # Overall V1 readiness score
            v1_readiness_indicators = [
                analysis["v1_compatible"],
                analysis["has_complete_flow"],
                analysis["adequate_spontaneity"],
                analysis["adequate_culture"],
                analysis["all_discussions_enhanced"]
            ]
            analysis["v1_readiness_score"] = sum(v1_readiness_indicators)
            analysis["ready_for_v1_script_generation"] = analysis["v1_readiness_score"] >= 4
            analysis["perfect_v1_outline"] = analysis["v1_readiness_score"] == 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze final quality"}

    def create_v1_summary(self, final_json):
        """
        Create a summary showing V1 compatibility
        """
        try:
            import json
            final_outline = json.loads(final_json)
            
            summary = {}
            
            # Extract key V1 elements
            conv_flow = final_outline.get("conversation_flow", {})
            summary["intro1_ready"] = bool(conv_flow.get("intro1", {}).get("opening_line"))
            summary["main_points_count"] = len(conv_flow.get("main_discussion", []))
            summary["spontaneous_elements"] = len(final_outline.get("spontaneous_moments", {}).get("natural_interruptions", []))
            summary["cultural_references"] = len(final_outline.get("cultural_context", {}).get("proverbs_sayings", []))
            summary["personality_defined"] = bool(final_outline.get("personality_interactions", {}).get("natural_chemistry"))
            
            # V1 script generation readiness
            summary["v1_script_ready"] = all([
                summary["intro1_ready"],
                summary["main_points_count"] >= 3,
                summary["spontaneous_elements"] >= 2,
                summary["personality_defined"]
            ])
            
            return summary
            
        except:
            return {"error": "Could not create V1 summary"}



In [12]:
polisher = FinalPolishEnhancer(deployment, "gpt-4o")
final_outline = polisher.add_final_polish(topic, information, classification_result, persona_result, style_enhanced_result)
print(final_outline)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "ليلى أحمد",
      "background": "صحفية ومقدمة برامج ثقافية مهتمة بالتكنولوجيا والهوية العربية.",
      "speaking_style": "تتحدث بطريقة تحليلية واضحة وتطرح أسئلة ذكية."
    },
    "guest": {
      "name": "د. سامي الحسن",
      "background": "أستاذ جامعي وخبير في الذكاء الاصطناعي وتطبيقاته في العالم العربي.",
      "speaking_style": "يشرح الأفكار بطريقة علمية مبسطة وموجهة للجمهور."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحبًا بكم في حلقة جديدة من بودكاستنا حيث نناقش القضايا التي تؤثر على ثقافتنا وهويتنا.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع في غاية الأهمية: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هناك تساؤلات حول تأثيره على لغتنا وقيمنا. هل يمكن لهذه التقنيات أن تكون أداة لتعزيز هويتنا أم أنها خطر يهد

In [ ]:
class HybridScriptGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment
        
        # V1-style rich dialogue examples
        self.arabic_dialogue_styles = {
            "حواري": {
                "host_example": "أحمد: يا أهلاً نور! كيف الحال؟ اممم... قوليلي، شو اللي خلاكِ تدخلي هذا المجال؟ <happy>",
                "guest_example": "نور: أهلاً أحمد! الله يعطيك العافية... يعني بصراحة، هاي قصة طويلة شوي [pause: 2s] بس باختصار، كنت أشوف المشاكل حولي وأقول: ليش ما نحلها بالتقنية؟"
            },
            "تعليمي": {
                "host_example": "أحمد: اممم... نور، ممكن تشرحي لنا بطريقة بسيطة، يعني شلون تشتغل هاي التقنية؟ <happy>",
                "guest_example": "نور: طبعاً أحمد! يعني... اههه كيف أشرح [pause: 2s] تخيل إنك عندك نظام ذكي جداً، بس هذا النظام مش إنسان، هو كمبيوتر! واو صح؟"
            },
            "ترفيهي": {
                "host_example": "أحمد: هاي نور! <happy> قوليلي، إيش أغرب موقف صار معكِ في الشغل؟ يعني شي يضحك؟",
                "guest_example": "نور: ههههه واو أحمد! بصراحة مواقف كثيرة... اممم مرة كنت أجرب البرنامج وفجأة [pause: 2s] خلاص ما عاد يشتغل! قعدت أصرخ: وين راح كودي؟! <surprise>"
            },
            "تحليلي": {
                "host_example": "أحمد: نور، بناءً على الإحصائيات الحديثة، وش رايكِ في التحديات الرئيسية اللي تواجه هذا المجال؟",
                "guest_example": "نور: سؤال ممتاز أحمد... يعني إذا نتكلم بشكل تحليلي، عندنا ثلاث تحديات أساسية [pause: 2s] أولها التقنية، ثانيها التمويل، وثالثها... اممم التقبل المجتمعي"
            }
        }
        
        # V1-style comprehensive fillers guide
        self.fillers_guide = """
Use natural conversation fillers with moderate density:
- Thinking: اممم، اههه، يعني كيف أقول، خلاص، شوف
- Confirmation: طبعاً، تماماً، بالضبط، صحيح، أكيد
- Hesitation: يعنييييي، يعني، اه ما أدري، مش عارف
- Emotion: واو، يا الله، ما شاء الله، الله يعطيك العافية
- Connection: بس، لكن، وبعدين، يا أخي، اسمع، انتبه
- Light Gulf dialect: شلون، وش رايك، زين، ماشي الحال، الله يعافيك
"""

    def generate_enhanced_intro(self, topic, final_outline_result, optimal_style):
        """
        Step 1: Generate enhanced intro (intro1 + intro2) - V1 enhancer style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        intro1 = conv_flow.get("intro1", {})
        intro2 = conv_flow.get("intro2", {})
        personas = outline.get("personas", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        prompt = f"""
You are an expert in writing natural and engaging Arabic podcast introductions.

Task: Write a complete natural introduction (intro1 + intro2) that combines welcome, topic introduction, and guest introduction.

Style examples required:
Host: {style_examples["host_example"]}
Guest: {style_examples["guest_example"]}

Introduction elements from outline:
Opening line: {intro1.get('opening_line', '')}
Podcast introduction: {intro1.get('podcast_introduction', '')}
Episode hook: {intro1.get('episode_hook', '')}
Spontaneity elements: {intro1.get('spontaneity_elements', [])}

Topic introduction: {intro2.get('topic_introduction', '')}
Guest welcome: {intro2.get('guest_welcome', '')}
Guest bio highlight: {intro2.get('guest_bio_highlight', '')}
Cultural connections: {intro2.get('cultural_connections', [])}

Available cultural references: {cultural_context.get('proverbs_sayings', [])}

{self.fillers_guide}

Introduction requirements:
1. Start with host {host_name} speaking alone (intro1)
2. Then bring guest {guest_name} into the conversation (intro2)
3. Use 70% MSA and 30% light Gulf touches
4. Add natural fillers with moderate density
5. Naturally incorporate spontaneous and cultural elements
6. Make smooth transition from host alone to dialogue with guest
7. Use actual character names
8. Add natural interactions: <happy>, <pause: 2s>, <overlap>
9. Make introduction engaging with 2-3 minutes duration
10. End with natural transition to main discussion
11. Write all dialogue content in Modern Standard Arabic with light dialectal touches

Dialogue format:
المقدم: [text]
الضيف: [text]

Write the complete introduction:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in writing natural and engaging Arabic podcast introductions. Style: {optimal_style}. Always write dialogue content in Modern Standard Arabic with light dialectal touches."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        
        return response.choices[0].message.content

    def generate_flowing_main_discussion(self, topic, final_outline_result, optimal_style, intro_dialogue):
        """
        Step 2: Generate flowing main discussion (all 3 points) - V1 dialogue generator style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        main_discussion = conv_flow.get("main_discussion", [])
        personas = outline.get("personas", {})
        spontaneous_moments = outline.get("spontaneous_moments", {})
        personality_interactions = outline.get("personality_interactions", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        # Build main points summary
        points_summary = ""
        for i, point in enumerate(main_discussion):
            points_summary += f"""
Point {i+1}: {point.get('point_title', '')}
- Personal angle: {point.get('personal_angle', '')}
- Spontaneous triggers: {point.get('spontaneous_triggers', [])}
- Disagreement points: {point.get('disagreement_points', '')}
- Cultural references: {point.get('cultural_references', [])}
- Emotional triggers: {point.get('emotional_triggers', '')}
"""
        
        prompt = f"""
You are an expert in writing natural flowing dialogues for Arabic podcasts.

Task: Write the main discussion as a natural flowing dialogue covering all three points.

Style examples required:
Host: {style_examples["host_example"]}
Guest: {style_examples["guest_example"]}

Context - end of introduction:
{intro_dialogue[-300:]}

Points to cover:
{points_summary}

Available spontaneous elements:
- Natural interruptions: {spontaneous_moments.get('natural_interruptions', [])}
- Emotional reactions: {spontaneous_moments.get('emotional_reactions', [])}
- Personal stories: {spontaneous_moments.get('personal_stories', [])}
- Humorous moments: {spontaneous_moments.get('humorous_moments', [])}

Character dynamics:
- Natural chemistry: {personality_interactions.get('natural_chemistry', '')}
- Tension points: {personality_interactions.get('tension_points', '')}
- Collaboration moments: {personality_interactions.get('collaboration_moments', '')}

{self.fillers_guide}

Discussion requirements:
1. Start with natural transition from introduction
2. Cover all three points with flowing natural dialogue
3. Don't strictly follow point order - let dialogue flow naturally
4. Naturally incorporate spontaneous and cultural elements
5. Add disagreement points and healthy debate
6. Use personal stories and cultural references
7. Make {host_name} and {guest_name} interact with their personalities
8. Add funny and surprising moments
9. Use 70% MSA and 30% Gulf touches
10. Duration: 6-7 minutes of natural dialogue
11. End with natural lead-in to conclusion
12. Write all dialogue content in Modern Standard Arabic with light dialectal touches

Dialogue format:
المقدم: [text]
الضيف: [text]

Write the complete discussion:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in writing flowing and natural dialogues. Style: {optimal_style}. Always write dialogue content in Modern Standard Arabic with light dialectal touches."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        
        return response.choices[0].message.content

    def generate_natural_closing(self, topic, final_outline_result, optimal_style, intro_dialogue, main_dialogue):
        """
        Step 3: Generate natural closing (conclusion + outro) - V1 enhancer style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        closing = conv_flow.get("closing", {})
        personas = outline.get("personas", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        prompt = f"""
You are an expert in writing impactful and natural Arabic podcast closings.

Task: Write a natural and impactful closing (conclusion + outro) for the episode.

Style examples required:
Host: {style_examples["host_example"]}

Context - end of discussion:
{main_dialogue[-300:]}

Closing elements from outline:
Main takeaways: {closing.get('conclusion', {}).get('main_takeaways', '')}
Guest final message: {closing.get('conclusion', {}).get('guest_final_message', '')}
Host closing thoughts: {closing.get('conclusion', {}).get('host_closing_thoughts', '')}

Guest appreciation: {closing.get('outro', {}).get('guest_appreciation', '')}
Audience thanks: {closing.get('outro', {}).get('audience_thanks', '')}
Call to action: {closing.get('outro', {}).get('call_to_action', '')}
Final goodbye: {closing.get('outro', {}).get('final_goodbye', '')}

Cultural references: {cultural_context.get('proverbs_sayings', [])}

{self.fillers_guide}

Closing requirements:
1. Start with natural transition from discussion
2. Summarize key points naturally, not mechanically
3. Have {guest_name} deliver impactful final message
4. Have {host_name} add his closing thoughts
5. Add sincere and warm thanks to guest and listeners
6. Include natural call for engagement
7. Use cultural touch in farewell
8. Make ending leave positive impact
9. Use 70% MSA and 30% Gulf touches
10. Duration: 1-2 impactful minutes
11. Write all dialogue content in Modern Standard Arabic with light dialectal touches

Dialogue format:
المقدم: [text]
الضيف: [text]

Write the complete closing:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"You are an expert in writing impactful and natural closings. Style: {optimal_style}. Always write dialogue content in Modern Standard Arabic with light dialectal touches."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content

    def generate_complete_script(self, topic, final_outline_result):
        """
        Generate complete script using 3-step hybrid approach
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        optimal_style = "تعليمي"  # Default
        if "language_style" in outline:
            formality = outline["language_style"].get("formality_level", "")
            if "حواري" in formality or "ودي" in formality:
                optimal_style = "حواري"
            elif "تحليلي" in formality:
                optimal_style = "تحليلي"
            elif "مرح" in formality or "ترفيهي" in formality:
                optimal_style = "ترفيهي"
        
        print(f"Starting script generation with style: {optimal_style}")
        print("=" * 50)
        
        print(" Step 1: Generating enhanced introduction...")
        intro_dialogue = self.generate_enhanced_intro(topic, final_outline_result, optimal_style)
        print("Introduction completed")
        
        print(" Step 2: Generating flowing discussion...")
        main_dialogue = self.generate_flowing_main_discussion(topic, final_outline_result, optimal_style, intro_dialogue)
        print(" Main discussion completed")
        
        print(" Step 3: Generating natural closing...")
        closing_dialogue = self.generate_natural_closing(topic, final_outline_result, optimal_style, intro_dialogue, main_dialogue)
        print("Closing completed")
        
        complete_script = f"""=== المقدمة ===
{intro_dialogue}

=== النقاش الرئيسي ===
{main_dialogue}

=== الختام ===
{closing_dialogue}"""
        
        print("🎉 Complete script generated successfully!")
        
        return {
            "intro": intro_dialogue,
            "main_discussion": main_dialogue,
            "closing": closing_dialogue,
            "complete_script": complete_script,
            "script_length": len(complete_script),
            "estimated_duration": "10 دقائق تقريباً",
            "style_used": optimal_style
        }



In [14]:
script_generator = HybridScriptGenerator(deployment, "gpt-4o")
script_result = script_generator.generate_complete_script(topic, final_outline)
print(script_result["complete_script"])

🎙️ Starting script generation with style: تحليلي
📝 Step 1: Generating enhanced introduction...
✅ Introduction completed
📝 Step 2: Generating flowing discussion...
✅ Main discussion completed
📝 Step 3: Generating natural closing...
✅ Closing completed
🎉 Complete script generated successfully!
=== المقدمة ===
المقدمة: ليلى أحمد: مرحبًا بكم جميعًا في حلقة جديدة من بودكاستنا حيث نناقش القضايا التي تؤثر على ثقافتنا وهويتنا بطريقة تحاكي واقعنا وتُلهمنا. أمم، شلون أقول، اليوم موضوعنا جدًا مثير للاهتمام... الذكاء الاصطناعي والهوية العربية! شوفوا، كلنا نلاحظ كيف الذكاء الاصطناعي صار جزءًا من حياتنا اليومية، أليس كذلك؟ يعني من التطبيقات، إلى القرارات، وحتى المحتوى الرقمي اللي نتعامل معه يوميًا. <pause: 1s> شخصيًا، كثيرًا ما أسأل نفسي: هل هالتقنيات المتطورة ممكن تكون أداة لتعزيز هويتنا الثقافية؟ ولا بالعكس، ممكن تشكل خطر يهدد اللغة والقيم اللي نحاول نحافظ عليها؟

أههه، طبعًا، الموضوع مو بسيط أبدًا، وفيه جوانب تقنية وثقافية وحتى اجتماعية تحتاج لتحليل عميق. واليوم، يسعدني جدًا أننا ما رح نناقش هذا 